<a href="https://colab.research.google.com/github/theprincejr/Prime_Numbers/blob/master/Telegram_Downloader_Script_(Remux_to_Temp_Folder_and_Replace).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3

from pyrogram import Client, filters, enums # Import enums for parse mode
from tqdm import tqdm # Keep tqdm for potential future use or if needed elsewhere
import os
# Removed import time # Removed unused import time
import subprocess
from dotenv import load_dotenv
import asyncio
import logging
import sys # Import sys for exiting on critical errors
# Removed import psutil # Removed psutil as system stats are removed
# Removed import hashlib # Removed hashlib as hash checking is removed
import requests # Added to trigger Emby scan via API (though now handled by scanner)
from contextlib import nullcontext # Keep nullcontext for potential future use
import shutil # Added for moving files
import re # Import re for sanitizing filenames
import time # Re-imported time for duration calculation

# === Load Environment Variables ===
load_dotenv() # Load environment variables from .env file

# === Configuration from Environment Variables (pre-logging setup) ===
# Use os.getenv() to read values from environment variables.
# Provide default values where appropriate, or raise errors for required variables.

# Telegram Setup (Read first for logging setup)
# API_ID and API_HASH are required. Exit if not found.
api_id_str = os.getenv('API_ID')
api_hash = os.getenv('API_HASH') # Corrected case to match common convention

# Session file path (defaults to a relative path within the project directory)
SESSION_FILE_PATH = os.getenv('SESSION_FILE', 'my_account.session') # Updated default path

# Download Paths (defaults if not set - these should remain on the SSD)
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_DIR', '/mnt/ssd/telegram_downloads')
OTHER_FOLDER = os.path.join(DOWNLOAD_FOLDER, "others")

# Temporary directory for remuxing (MUST be outside DOWNLOAD_FOLDER)
REMUX_TEMP_DIR = os.getenv('REMUX_TEMP_DIR', '/mnt/ssd/telegram_remux_temp') # New environment variable

# Log File Path (defaults to a relative path within the project directory)
LOG_FILE = os.getenv('LOG_FILE', 'telegram_downloader.log') # Updated default path

# Chat ID for filtering messages (defaults to 'me' if not set)
# Pyrogram's filters.chat() can take string usernames ('me', '@username') or integer IDs.
chat_filter_id_env = os.getenv('CHAT_ID', 'me')

# Convert chat_filter_id to integer if it looks like one, otherwise keep as string
try:
    # Attempt to convert to int, but handle empty string or non-numeric
    if chat_filter_id_env is not None and str(chat_filter_id_env).strip().isdigit():
        chat_filter_id = int(str(chat_filter_id_env).strip())
    else:
        chat_filter_id = chat_filter_id_env # Keep as string if not an integer or is None/empty
except ValueError:
    # This block should ideally not be reached due to the isdigit() check, but as a safeguard
    chat_filter_id = chat_filter_id_env # Keep as string on unexpected ValueError


# Path to the downloader status file (defaults to a relative path within the project directory)
DOWNLOADER_STATUS_FILE = os.getenv('DOWNLOADER_STATUS_FILE', '/home/pi/project_d/telegram_downloader/downloader_status.txt') # Updated default path

# Removed NOTIFICATION_CHAT_ID as notifications are not sent from here

# Emby Server Details (Required for triggering scan - now handled by scanner script)
# Keeping these variables for the trigger_emby_scan_from_downloader function if needed elsewhere
EMBY_URL = os.getenv('EMBY_URL')
EMBY_API_KEY = os.getenv('EMBY_API_KEY')


# === Logging Setup ===
# Configure logging AFTER reading LOG_FILE from environment, but BEFORE critical checks
# Ensure the log directory exists (will be the current working directory if path is relative)
log_dir = os.path.dirname(LOG_FILE)
if log_dir and log_dir != '' and not os.path.exists(log_dir):
    os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level=logging.INFO, # Keep INFO level for general operation
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, mode='a', encoding='utf-8'),
        logging.StreamHandler(sys.stdout) # Use sys.stdout for stream handler
    ]
)
logging.info("Logging configured.") # Add a log message to confirm logging is active

# === Configuration Validation (post-logging setup) ===
# Now perform checks and exit if necessary, ensuring errors are logged to file

if not api_id_str:
    logging.error("FATAL: API_ID environment variable not set.")
    sys.exit(1)
if not api_hash:
    logging.error("FATAL: API_hash environment variable not set.")
    sys.exit(1)

try:
    api_id = int(api_id_str)
except ValueError:
    logging.error(f"FATAL: API_ID environment variable '{api_id_str}' is not a valid integer.")
    sys.exit(1)

# Ensure download folders exist AFTER reading DOWNLOAD_FOLDER from environment
# Also ensure the parent directory of the session file exists if it's not home
# Note: If SESSION_FILE_PATH is a relative path like 'my_account.session',
# os.path.dirname() will return '', and os.makedirs('') does nothing, which is correct.
session_dir = os.path.dirname(SESSION_FILE_PATH)
if session_dir and session_dir != '' and not os.path.exists(session_dir):
    os.makedirs(session_dir, exist_ok=True)
    logging.info(f"Created session file directory: {session_dir}")


os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)
os.makedirs(OTHER_FOLDER, exist_ok=True)
# Ensure the temporary remux directory exists
os.makedirs(REMUX_TEMP_DIR, exist_ok=True)

logging.info(f"Download folder set to: {DOWNLOAD_FOLDER}")
logging.info(f"Other files folder set to: {OTHER_FOLDER}")
logging.info(f"Remux temporary folder set to: {REMUX_TEMP_DIR}") # Log the new directory
logging.info(f"Session file path set to: {SESSION_FILE_PATH}")
logging.info(f"Chat filter ID set to: {chat_filter_id}")
logging.info(f"Downloader status file set to: {DOWNLOADER_STATUS_FILE}")

# Validate Emby configuration if needed - now only used for the trigger function if called
# This check is still useful to know if the variables are loaded correctly in the downloader's env
# Changed logging level from warning to debug for this check
if not EMBY_URL or not EMBY_API_KEY:
    logging.debug("Emby URL or API Key not set in downloader's environment. Manual Emby scan trigger from downloader will be skipped.")
else:
    logging.info(f"Emby URL set in downloader's environment: {EMBY_URL}")


MAX_RETRIES = 3
RETRY_TAIL_SIZE = 3 * 1024 * 1024 # Last 3MB for integrity retry

# Initialize the Pyrogram client
# Use the variables read from the environment
app = Client(SESSION_FILE_PATH, api_id=api_id, api_hash=api_hash)
logging.debug(f"[DEBUG] Pyrogram client initialized.")


# Dictionary to store filename and their size immediately after initial download
# This is for duplicate checking based on pre-remux size during the current run.
# This dictionary is reset each time the script starts.
downloaded_file_sizes = {}

# Removed Variable to track the current download status
# Removed Variable to track the current download filename

def is_video(message):
    # Check if the message has a document and its mime type starts with video/
    # This is a good check.
    # Also check if it's a native video object
    return (message.document and message.document.mime_type and message.document.mime_type.startswith("video/")) or message.video

# Removed the verify_video_ffmpeg function as requested.

# Removed calculate_file_hash function
# Removed notify_duplicate function

# === Emby Scan Trigger Function (called from downloader - now only for manual trigger if needed) ===
# This function is no longer called automatically on download start/end in this version
# Keeping the function definition in case it's useful for debugging or future manual triggers
def trigger_emby_scan_from_downloader():
    """Sends a request to the Emby API to trigger a library scan."""
    # This check is important here - it's why the warning appears in the log
    # Changed logging level from warning to debug for this check
    if not EMBY_URL or not EMBY_API_KEY:
        logging.debug("Emby URL or API Key not set in downloader's environment. Skipping Emby scan trigger.")
        return

    # Emby API endpoint for library refresh, includes api_key in the URL
    # Assuming the endpoint is /Library/Refresh based on typical Emby API
    scan_endpoint = f"{EMBY_URL}/Library/Refresh?api_key={EMBY_API_KEY}"

    # Headers might not be strictly necessary for this endpoint with API key in URL,
    # but including Content-Type is good practice.
    headers = {
        'Content-Type': 'application/json'
    }

    logging.info(f"Attempting to trigger Emby scan via API from downloader: {scan_endpoint.split('?')[0]}...") # Log URL without API key

    try:
        # Send a POST request to the refresh endpoint
        response = requests.post(scan_endpoint, headers=headers)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        logging.info(f"Emby scan triggered successfully from downloader. Response status code: {response.status_code}")
        # Emby API /Library/Refresh typically returns 204 No Content on success.

    except requests.exceptions.RequestException as e:
        logging.error(f"Downloader failed to trigger Emby scan via API. Error: {e}")
        # Log response details if available for debugging
        if hasattr(e, 'response') and e.response is not None:
            logging.error(f"Emby API response status: {e.response.status_code}")
            logging.error(f"Emby API response body: {e.response.text}")
    except Exception as e:
        logging.error(f"An unexpected error occurred while triggering Emby scan from downloader: {e}")


# Modified remux_video_ffmpeg to accept input and output paths
def remux_video_ffmpeg(input_path, output_path):
    """Remuxes a video file to MKV format using FFmpeg."""
    logging.info(f"[REMUX] Starting remux of '{os.path.basename(input_path)}' to MKV.")
    # Update status file for remuxing (multi-line)
    update_status_file(f"Status: Remuxing {os.path.basename(input_path)}")

    # FFmpeg command to remux (copying streams without re-encoding)
    # -i: input file
    # -c copy: copy all streams (video, audio, subtitles) without re-encoding
    # -map 0: include all streams from the input
    # -y: overwrite output file without asking
    # output_path: the output MKV file
    command = [
        'ffmpeg',
        '-i', input_path,
        '-c', 'copy',
        '-map', '0',
        '-y',
        output_path
    ]

    try:
        # Run the FFmpeg command
        process = subprocess.run(command, capture_output=True, text=True, check=True)
        # Keep FFmpeg output at DEBUG level, so it doesn't clutter INFO logs
        logging.debug(f"[REMUX] FFmpeg stdout:\n{process.stdout}")
        logging.debug(f"[REMUX] FFmpeg stderr:\n{process.stderr}")
        logging.info(f"[REMUX] Successfully remuxed: {os.path.basename(input_path)}")

        # Remove the original file after successful remux - HANDLED AFTER REMUX CALL

        return True # Remux successful

    except subprocess.CalledProcessError as e:
        logging.error(f"[REMUX] FFmpeg remux failed for '{os.path.basename(input_path)}'. Error: {e}")
        logging.error(f"[REMUX] FFmpeg stdout:\n{e.stdout}")
        logging.error(f"[REMUX] FFmpeg stderr:\n{e.stderr}")
        # Update status file for remux failure (multi-line)
        update_status_file(f"Status: Remux Failed for {os.path.basename(input_path)}\nError: {e.stderr.strip()}")
        # Clean up the incomplete remuxed file if it exists - HANDLED AFTER REMUX CALL
        return False # Remux failed
    except FileNotFoundError:
        logging.error("[REMUX] FFmpeg command not found. Is FFmpeg installed and in PATH?")
        # Update status file for FFmpeg not found (multi-line)
        update_status_file(f"Status: Remux Failed\nError: FFmpeg not found.")
        return False
    except Exception as e:
        logging.error(f"[REMUX] An unexpected error occurred during remux: {e}", exc_info=True)
        # Update status file for unexpected remux error (multi-line)
        update_status_file(f"Status: Remux Failed for {os.path.basename(input_path)}\nError: {e}")
        # Clean up the incomplete remuxed file if it exists - HANDLED AFTER REMUX CALL
        return False

# Reverted notify_completion to send to 'me' chat
async def notify_completion(client, path, duration, remux_status='Skipped'):
    """Sends a completion message to the 'me' chat."""
    try:
        # Ensure the file exists before trying to get its size
        if not os.path.exists(path):
            logging.warning(f"[NOTIFY] File not found for completion notification: {os.path.basename(path)}")
            size_bytes = 0
        else:
            size_bytes = os.path.getsize(path)

        size_mb = size_bytes / (1024 * 1024)
        # Avoid division by zero if duration is 0 (very fast download)
        avg_speed = size_mb / duration if duration > 0 else 0 # Corrected division by zero case

        minutes = int(duration // 60)
        seconds = int(duration % 60)

        # Add remux status to the message
        remux_line = f"🔄 Remux: `{remux_status}`"

        msg = (
            f"✅ *Download complete*\n"
            f"📁 `{os.path.basename(path)}`\n"
            f"📦 Size: `{size_mb:.2f} MB`\n"
            f"⏱ Time: `{minutes}m {seconds}s`\n"
            f"🚀 Speed: `{avg_speed:.2f} MB/s`\n"
            f"{remux_line}" # Include the remux status line
        )
        logging.info(f"[NOTIFY] Attempting to send completion message for: {os.path.basename(path)}")
        # Use markdown parse_mode for formatting in Telegram message
        # Corrected parse_mode to use enums.ParseMode.MARKDOWN
        await client.send_message("me", msg, parse_mode=enums.ParseMode.MARKDOWN)
        logging.info(f"[NOTIFY] Completion message sent successfully for: {os.path.basename(path)}")
    except Exception as e:
        logging.error(f"[NOTIFY] Failed to send completion notification for {os.path.basename(path)}. Error: {e}")

def truncate_last_bytes(file_path, bytes_to_trim):
    """Truncates the end of a file. Useful for retrying partial downloads."""
    try:
        size = os.path.getsize(file_path)
        if size > bytes_to_trim:
            # Open in read+write binary mode
            with open(file_path, "rb+") as f:
                # Seek to the position before the bytes to trim
                f.seek(size - bytes_to_trim)
                f.truncate() # Truncate the file at the current position
            logging.info(f"[TRUNCATE] Trimmed last {bytes_to_trim} bytes of: {file_path}")
        elif size > 0:
             # If file is smaller than bytes_to_trim but not empty, truncate to 0
             logging.warning(f"[TRUNCATE] File size {size} is smaller than {bytes_to_trim}. Truncating to 0: {file_path}")
             with open(file_path, "wb") as f:
                 pass # Open and close in write mode to clear content
        else:
            logging.info(f"[TRUNCATE] File is empty or doesn't exist, no truncation needed: {file_path}")

    except FileNotFoundError:
        logging.warning(f"[TRUNCATE] File not found, cannot truncate: {file_path}")
    except Exception as e:
        logging.error(f"[TRUNCATE] Failed to truncate {file_path}. Error: {e}")

# === Status File Management ===
def update_status_file(status_message):
    """Writes the current status to the status file."""
    try:
        # Ensure the directory for the status file exists
        status_file_dir = os.path.dirname(DOWNLOADER_STATUS_FILE)
        if status_file_dir and status_file_dir != '' and not os.path.exists(status_file_dir):
            os.makedirs(status_file_dir, exist_ok=True)

        with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f:
            f.write(status_message)
        # Changed this logging level from debug to info for cleaner combined log
        # Log the status message on a single line for the combined log file
        # Perform the replace outside the f-string expression
        single_line_status = status_message.replace('\n', ', ')
        logging.info(f"Status file updated: {single_line_status}") # <--- Corrected line
    except Exception as e:
        logging.error(f"Failed to write to status file {DOWNLOADER_STATUS_FILE}: {e}")

def clear_status_file():
    """Clears the content of the status file."""
    try:
        # Ensure the directory for the status file exists
        status_file_dir = os.path.dirname(DOWNLOADER_STATUS_FILE)
        if status_file_dir and status_file_dir != '' and not os.path.exists(status_file_dir):
            os.makedirs(status_file_dir, exist_ok=True)

        with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f:
            f.write("Downloader script started.") # Write a default status
        logging.info("Status file cleared.") # Changed logging level to info
    except Exception as e:
        logging.error(f"Failed to clear status file {DOWNLOADER_STATUS_FILE}: {e}")

# === Progress Bar Hook ===
# This function is called periodically during download by client.stream_media
# NOTE: This function might not be called by stream_media in some Pyrogram versions.
# If progress updates stop, this is the likely reason.
# Re-added time import for this function
# def progress_hook(current, total, client, message, start_time):
#     """Updates the status file and potentially the Telegram message with download progress."""
#     # Update every 5% or on completion, and ensure enough time has passed or enough data downloaded
#     # Added a time check (at least 1 second) and data check (at least 10KB) to avoid excessive updates
#     current_time = time.time()
#     # Check if it's time to update or if it's the very beginning (current == 0) or the very end (current == total)
#     if current == 0 or current == total or \
#        (current_time - progress_hook.last_update_time > 1 or (current - progress_hook.last_update_bytes) > 10 * 1024):

#         elapsed_time = current_time - start_time
#         if elapsed_time > 0 and total > 0: # Avoid division by zero if total is 0
#             speed = current / elapsed_time # bytes per second
#             # Calculate ETA only if speed is reasonable and current is not 0
#             if speed > 1024 and current > 0:
#                  eta_seconds = (total - current) / speed
#                  # Format ETA nicely (e.g., 1m 30s, 5h 10m)
#                  if eta_seconds < 60:
#                       eta_str = f"{int(eta_seconds)}s"
#                  elif eta_seconds < 3600:
#                       eta_str = f"{int(eta_seconds // 60)}m {int((eta_seconds % 3600) // 60)}s"
#                  else:
#                       eta_str = f"{int(eta_seconds // 3600)}h {int((eta_seconds % 3600) // 60)}m"
#             else:
#                  eta_str = "Calculating..."

#             # Format speed nicely (e.g., 1.2 MB/s, 500 KB/s)
#             if speed > 1024*1024:
#                  speed_str = f"{speed / (1024*1024):.2f}MB/s"
#             elif speed > 1024:
#                  speed_str = f"{speed / 1024:.2f}KB/s"
#             else:
#                  speed_str = f"{speed:.2f}B/s"

#             # Calculate overall progress percentage
#             progress_percent = (current / total) * 100 if total > 0 else 0

#             # Get filename from document or video media
#             file_name = message.document.file_name if message.document else message.video.file_name if message.video else 'File'

#             # Format the status message for the status file (multi-line for the file)
#             status_message_file = (
#                 f"Downloading: {file_name}\n"
#                 f"Status: Downloading\n"
#                 f"Progress: {progress_percent:.1f}%\n"
#                 f"Speed: {speed_str}\n"
#                 f"ETA: {eta_str}" # Include ETA in the status file
#             )

#             # Format the status message for the log (single line)
#             status_message_log = (
#                 f"Downloading: {file_name}, "
#                 f"Status: Downloading, "
#                 f"Progress: {progress_percent:.1f}%, "
#                 f"Speed: {speed_str}, "
#                 f"ETA: {eta_str}"
#             )


#             # Write the multi-line status to the status file
#             try:
#                 status_file_dir = os.path.dirname(DOWNLOADER_STATUS_FILE)
#                 if status_file_dir and status_file_dir != '' and not os.path.exists(status_file_dir):
#                     os.makedirs(status_file_dir, exist_ok=True)

#                 with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f:
#                     f.write(status_message_file)
#             except Exception as e:
#                 logging.error(f"Failed to write to status file {DOWNLOADER_STATUS_FILE}: {e}")

#             # Log the single-line status using logging (goes to stderr now)
#             # Corrected this line to use status_message_log instead of single_line_status
#             logging.info(f"Status file updated: {status_message_log}")


#             # Update last update time and bytes for debounce
#             progress_hook.last_update_time = current_time
#             progress_hook.last_update_bytes = current

#         # Optional: Update the Telegram message itself with progress (can be noisy)
#         # try:
#         #     # Avoid editing too frequently to prevent FloodWait
#         #     if int(current * 100 / total) % 10 == 0 or current == total:
#         #         await message.edit_text(f"Downloading: {current * 100 / total:.1f}%")
#         # except FloodWait as e:
#         #     logging.warning(f"FloodWait: Sleeping for {e.value} seconds.")
#         #     time.sleep(e.value)
#         # except RPCError as e:
#         #      logging.error(f"RPC Error while editing message: {e}")
#         # except Exception as e:
#         #      logging.error(f"Error updating Telegram message: {e}")

# # Initialize debounce variables for the progress_hook function
# progress_hook.last_update_time = 0
# progress_hook.last_update_bytes = 0

# Function to sanitize filenames
def sanitize_filename(filename):
    """Sanitizes a string to be safe for use as a filename."""
    # Replace characters that are not alphanumeric, underscores, hyphens, spaces, or periods
    # with underscores. Also, remove leading/trailing spaces and periods.
    # This is a basic sanitization and might need adjustment based on specific filesystem requirements.
    sanitized = re.sub(r'[^\w\s\.\-]', '_', filename).strip()
    # Replace spaces with underscores for better compatibility
    sanitized = re.sub(r'\s+', '_', sanitized)
    # Remove leading/trailing underscores and periods that might result from sanitization
    sanitized = sanitized.strip('_.')
    # Ensure the filename is not empty after sanitization
    if not sanitized:
        sanitized = "sanitized_file"
    return sanitized


async def download_media(client, message):
    """Main asynchronous function to handle downloading a single message's media."""
    logging.debug(f"[DEBUG] Entering download_media function for message {message.id}.")

    media = message.document or message.video or message.audio or message.photo
    if not media:
        logging.warning(f"Message {message.id} has no supported media.")
        logging.debug(f"[DEBUG] Exiting download_media: No supported media.")
        return

    logging.debug(f"[DEBUG] Before determining filename.")
    # Determine filename and sanitize it
    filename = getattr(media, 'file_name', None)
    if not filename:
        # Fallback filename if file_name is not available, sanitize the file_id
        safe_id = ''.join(c for c in media.file_id if c.isalnum()) # Sanitize file_id
        filename = f"{safe_id}.bin" # Use .bin as a generic fallback extension
        logging.info(f"[INFO] Generated and sanitized fallback filename {filename} for message {message.id}")
    else:
         filename = sanitize_filename(filename) # Sanitize the provided filename
         logging.info(f"[INFO] Sanitized filename: {filename}")


    logging.debug(f"[DEBUG] Before determining folder and path.")
    # Determine download folder based on media type
    folder = DOWNLOAD_FOLDER if is_video(message) else OTHER_FOLDER
    path = os.path.join(folder, filename)
    total_size = media.file_size or 0 # Use 0 if file_size is None

    logging.debug(f"[DEBUG] Before os.makedirs.")
    # Ensure download folders exist (Corrected: Ensure DOWNLOAD_FOLDER exists)
    os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)
    os.makedirs(OTHER_FOLDER, exist_ok=True)
    # Ensure remux temp directory exists
    os.makedirs(REMUX_TEMP_DIR, exist_ok=True)


    logging.debug(f"[DEBUG] Before refined duplicate check.")
    # === Refined Duplicate Check ===
    if os.path.exists(path):
        current_file_size = os.path.getsize(path)
        # Check if we have a stored pre-remux size for this filename (from the current run)
        if filename in downloaded_file_sizes:
            stored_pre_remux_size = downloaded_file_sizes[filename]
            if current_file_size == stored_pre_remux_size and stored_pre_remux_size > 0:
                logging.info(f"[SKIP] Already complete (filename/stored size match): {filename}")
                # Update status file to show idle or last completed
                try:
                    with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f: # Added encoding
                        f.write(f"Idle (Last completed: {filename})")
                except Exception as e:
                    logging.error(f"Failed to write idle status to file: {e}")
                logging.debug(f"[DEBUG] Exiting download_media: Duplicate (stored size match).")
                return
            # If size doesn't match stored size, it's either a partial download or oversized relative to stored.
            # Proceed to resume/redownload logic below.
        # If no stored size (script restarted), check if current file size matches incoming message size.
        # This handles remuxed files from previous runs that match the original size.
        elif current_file_size == total_size and total_size > 0:
            logging.info(f"[SKIP] Already complete (filename/incoming size match from previous run): {filename}")
            # Store this size as the "pre-remux" size for future checks in this run
            downloaded_file_sizes[filename] = total_size
            # Update status file to show idle or last completed
            try:
                with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f: # Added encoding
                    f.write(f"Idle (Last completed: {filename})")
                except Exception as e:
                    logging.error(f"Failed to write idle status to file: {e}")
                logging.debug(f"[DEBUG] Exiting download_media: Duplicate (incoming size match).")
                return
        # If file exists but size doesn't match any complete condition,
        # and it's not a zero-size file, it's either a partial download or oversized.
        # The existing oversized check against incoming size is still relevant here if no stored size exists.
        elif current_file_size > total_size and total_size > 0:
            # This case is hit if file exists, no stored size, and current size > incoming size.
            logging.warning(f"[OVERSIZE] Existing file {path} is larger than expected (incoming size, no stored size). Redownloading.")
            try:
                os.remove(path) # Remove and redownload
                logging.info(f"[OVERSIZE] Removed oversized file: {path}")
            except Exception as remove_e:
                 logging.error(f"[OVERSIZE] Failed to remove oversized file {path}: {remove_e}")
                 # If removal fails, we can't proceed safely, maybe log a critical error or skip?
                 # For now, let's log and let the download attempt potentially fail again.
                 pass # Continue to download attempt

        # If file exists but size doesn't match any complete condition, proceed to resume logic.

    logging.debug(f"[DEBUG] After refined duplicate check. Before total_size == 0 check.")
    if total_size == 0:
        logging.warning(f"[ZERO_SIZE] Media reports zero size. Skipping download for {filename}.")
        # Update status file to show skipped
        try:
            with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f: # Added encoding
                f.write(f"Skipped (Zero size): {filename}")
        except Exception as e:
            logging.error(f"Failed to write skipped status to file: {e}")
        logging.debug(f"[DEBUG] Exiting download_media: Zero size media.")
        return
    # === End Refined Duplicate Check ===

    logging.debug(f"[DEBUG] Before initializing retries and last_exception.")
    retries = 0
    last_exception = None

    # Removed progress_hook related debounce variables as progress_hook is removed

    logging.debug(f"[DEBUG] Before determining use_tqdm.")
    # Determine if tqdm should be enabled (only if total_size > 0 and stdout is a tty)
    # TQDM prints to stderr by default, but we're logging to stdout, so check stdout
    use_tqdm = total_size > 0 and sys.stdout.isatty()
    logging.debug(f"[DEBUG] TQDM enabled: {use_tqdm}")

    logging.debug(f"[DEBUG] Entering retry loop for {filename}.")

    while retries < MAX_RETRIES:
        logging.debug(f"[DEBUG] Inside retry loop, before try block.")
        try:
            logging.debug(f"[DEBUG] Starting download attempt {retries + 1} for {filename}.")
            # Truncate before attempting resume download
            if os.path.exists(path):
                downloaded_so_far = os.path.getsize(path)
                if downloaded_so_far > 0:
                    logging.info(f"[RESUME] Resuming download for {filename} from {downloaded_so_far} bytes.")
                    truncate_last_bytes(path, RETRY_TAIL_SIZE)
                    logging.debug(f"[DEBUG] After truncate_last_bytes for resume.")


            # Corrected: Use await client.download_media as it's an async function
            start_time = time.time() # Use time.time()
            logging.debug(f"[DEBUG] Starting download using client.download_media for {filename}.")
            # Use the path variable directly as file_name argument
            downloaded_path = await client.download_media(
                message,
                file_name=path,
                # Removed progress and progress_args as download_media doesn't use them in this way
            )
            duration = time.time() - start_time # Calculate duration after download

            logging.debug(f"[DEBUG] client.download_media finished for {filename}. Returned path: {downloaded_path}")

            # Verify the downloaded_path is not None and exists
            if not downloaded_path or not os.path.exists(downloaded_path):
                 raise Exception(f"Download failed or file not found after download_media for {filename}")

            # Final size check after download finishes
            final_size = os.path.getsize(downloaded_path)
            logging.debug(f"[DEBUG] Final file size check. Expected: {total_size}, Got: {final_size}.")
            # Note: Telegram's reported size might not always exactly match the downloaded size,
            # especially for some media types or due to container overhead.
            # A strict equality check might be too brittle. Consider a small tolerance if needed.
            # For now, keeping strict check as per previous logic.
            if final_size != total_size:
                 logging.warning(f"[SIZE_MISMATCH] Downloaded file size mismatch for {filename}. Expected: {total_size}, Got: {final_size}")
                 # Depending on how critical exact size is, you might want to raise an error here
                 # or just log the warning. Keeping as a warning for now.


            # Store the size immediately after successful download (pre-remux)
            downloaded_file_sizes[filename] = final_size
            logging.info(f"[SIZE_STORE] Stored pre-remux size for {filename}: {final_size} bytes")
            logging.debug(f"[DEBUG] Size stored.")

            # --- Update Status File on Completion ---
            completion_status_file = f"Download Complete: {filename}"
            single_line_completion_status = completion_status_file.replace('\n', ', ')
            logging.info(f"Status file updated: {single_line_completion_status}")
            try:
                with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f_status:
                    f_status.write(completion_status_file)
                logging.debug(f"[DEBUG] Completion status written to file.")
            except Exception as e:
                logging.error(f"Failed to write completion status to file: {e}")
            # --- End Update Status File on Completion ---


            # Perform video remuxing if applicable
            remux_status = 'Skipped' # Initialize remux status
            final_file_path = downloaded_path # Assume original downloaded file is the final one initially
            logging.debug(f"[DEBUG] Checking if remux is needed for {filename}.")
            if os.path.exists(downloaded_path) and is_video(message):
                 logging.info(f"[PROCESS] Original message was a video or document with video extension. Attempting remux of '{os.path.basename(downloaded_path)}'.")
                 remux_status = 'Attempted'
                 logging.debug(f"[DEBUG] Remux needed. Remux status set to 'Attempted'.")

                 # Define the temporary output path in the REMUX_TEMP_DIR
                 # Use the original filename base with .mkv extension for the temporary file
                 temp_remux_output_filename = os.path.splitext(filename)[0] + '.mkv'
                 temp_remux_output_path = os.path.join(REMUX_TEMP_DIR, temp_remux_output_filename)
                 logging.debug(f"[DEBUG] Temporary remux output path: {temp_remux_output_path}")

                 # Call remux_video_ffmpeg to remux to the temporary directory
                 logging.debug(f"[DEBUG] Calling remux_video_ffmpeg.")
                 remux_success = remux_video_ffmpeg(downloaded_path, temp_remux_output_path)
                 logging.debug(f"[DEBUG] remux_video_ffmpeg returned: {remux_success}")

                 if remux_success:
                     remux_status = 'Successful'
                     logging.info(f"[REMUX] Remux successful. Moving '{os.path.basename(temp_remux_output_path)}' to '{os.path.basename(downloaded_path)}' (replacing original).")
                     logging.debug(f"[DEBUG] Remux successful. Attempting file move.")

                     # Move the successfully remuxed file from temp to the final download folder
                     try:
                         # Ensure the original downloaded file exists before attempting to remove it
                         logging.debug(f"[DEBUG] Checking if original downloaded file exists at {downloaded_path} before removal.")
                         if os.path.exists(downloaded_path):
                             logging.debug(f"[DEBUG] Original downloaded file exists. Removing {downloaded_path}.")
                             os.remove(downloaded_path)
                             logging.info(f"[REMUX] Removed original downloaded file '{os.path.basename(downloaded_path)}' after successful remux.")
                             logging.debug(f"[DEBUG] Original downloaded file removed.")
                         else:
                             logging.warning(f"[REMUX] Original downloaded file not found at {downloaded_path} before removal attempt after remux.")


                         # Move the remuxed file to the original file's location
                         logging.debug(f"[DEBUG] Moving remuxed file from {temp_remux_output_path} to {downloaded_path}.")
                         shutil.move(temp_remux_output_path, downloaded_path)
                         logging.info(f"[REMUX] Successfully moved remuxed file to final location: '{os.path.basename(downloaded_path)}'.")
                         logging.debug(f"[DEBUG] Remuxed file moved successfully.")
                         final_file_path = downloaded_path # The final file is now at the original download path

                     except Exception as move_e:
                         logging.error(f"[REMUX] Failed to move remuxed file from temp to final location. Error: {move_e}")
                         remux_status = 'Move Failed'
                         logging.debug(f"[DEBUG] File move failed. Remux status set to 'Move Failed'.")
                         # Clean up the incomplete remuxed file in the temp directory if it exists
                         logging.debug(f"[DEBUG] Checking if incomplete remuxed file exists at {temp_remux_output_path} after move failure.")
                         if os.path.exists(temp_remux_output_path):
                             logging.debug(f"[DEBUG] Incomplete remuxed file exists. Attempting cleanup.")
                             try:
                                 os.remove(temp_remux_output_path)
                                 logging.info(f"[REMUX] Cleaned up incomplete remuxed file in temp directory after move failure: {os.path.basename(temp_remux_output_path)}")
                                 logging.debug(f"[DEBUG] Incomplete remuxed file cleaned up.")
                             except Exception as cleanup_e:
                                 logging.error(f"[REMUX] Failed to clean up incomplete remuxed file in temp directory after move failure: {cleanup_e}")
                                 logging.debug(f"[DEBUG] Failed to clean up incomplete remuxed file.")
                         # The original file might still exist if the move failed before deletion, or it was deleted.
                         # The final_file_path remains the original path, but the file might not be there.
                         # The completion notification will check if the file exists.


                 else:
                     logging.error(f"[REMUX] Remux failed for {os.path.basename(downloaded_path)}. Keeping original file.")
                     logging.debug(f"[DEBUG] Remux failed. Keeping original file.")
                     # If remux failed, the status file is updated inside remux_video_ffmpeg
                     # Clean up the incomplete remuxed file in the temp directory if it exists
                     logging.debug(f"[DEBUG] Checking if incomplete remuxed file exists at {temp_remux_output_path} after remux failure.")
                     if os.path.exists(temp_remux_output_path):
                         logging.debug(f"[DEBUG] Incomplete remuxed file exists. Attempting cleanup.")
                         try:
                             os.remove(temp_remux_output_path)
                             logging.info(f"[REMUX] Cleaned up incomplete remuxed file in temp directory after remux failure: {os.path.basename(temp_remux_output_filename)}") # Use filename for log clarity
                             logging.debug(f"[DEBUG] Incomplete remuxed file cleaned up.")
                         except Exception as cleanup_e:
                             logging.error(f"[REMUX] Failed to clean up incomplete remuxed file in temp directory after remux failure: {cleanup_e}")
                             logging.debug(f"[DEBUG] Failed to clean up incomplete remuxed file.")
                     # final_file_path remains the original path, which should still exist
            else:
                logging.debug(f"[DEBUG] Remux not needed or original file not found at {downloaded_path}.")


            logging.info(f"[DEBUG] About to notify completion for {os.path.basename(final_file_path)}")

            # Notify completion, passing the remux status and the final file path
            # duration is already calculated after download
            logging.debug(f"[DEBUG] Calling notify_completion with path: {final_file_path}, duration: {duration}, remux_status: {remux_status}.")
            await notify_completion(client, final_file_path, duration, remux_status)
            logging.debug(f"[DEBUG] notify_completion finished.")


            logging.debug(f"[DEBUG] Download and processing successful for {filename}. Returning.")
            return # Exit the retry loop on success


        except Exception as e:
            logging.debug(f"[DEBUG] Exception caught in download_media try block: {e}")
            retries += 1
            logging.warning(f"[RETRY] Download/Processing for {filename} failed ({retries}/{MAX_RETRIES}): {e}")
            last_exception = e
            logging.debug(f"[DEBUG] Retries: {retries}, Last exception stored.")


            # --- Update Status File on Retry/Failure ---
            retry_status_file = f"Download Failed/Retrying: {filename} (Attempt {retries}/{MAX_RETRIES})\nError: {e}"
            single_line_retry_status = retry_status_file.replace('\n', ', ')
            logging.info(f"Status file updated: {single_line_retry_status}")
            try:
                with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f_status:
                    f_status.write(retry_status_file)
                logging.debug(f"[DEBUG] Retry/Failure status written to file.")
            except Exception as write_e:
                logging.error(f"Failed to write retry/failure status to file: {write_e}")
                logging.debug(f"[DEBUG] Failed to write retry/failure status to file.")
            # --- End Update Status File on Retry/Failure ---


            if retries < MAX_RETRIES:
                logging.debug(f"[DEBUG] Retries < MAX_RETRIES. Sleeping for {5 * retries} seconds.")
                await asyncio.sleep(5 * retries) # Exponential backoff
                logging.debug(f"[DEBUG] Finished sleeping. Continuing retry loop.")
            else:
                # This block is reached only if all retries fail
                logging.error(f"[FAILED] Gave up on {filename} after {MAX_RETRIES} retries.")
                logging.debug(f"[DEBUG] All retries failed. Attempting permanent failure notification.")
                # The status file is already updated with the last failure in the except block
                # Reverted to sending failure notification to 'me' chat
                try:
                    failure_msg = (
                        f"❌ *Download Failed Permanently*\n"
                        f"File: `{filename}`\n"
                        f"Error: `{last_exception}`" # Include the exception string
                    )
                    logging.debug(f"[DEBUG] Sending permanent failure notification.")
                    await client.send_message("me", failure_msg, parse_mode=enums.ParseMode.MARKDOWN)
                    logging.debug(f"[DEBUG] Permanent failure notification sent.")
                except Exception as notify_e:
                    logging.error(f"[NOTIFY] Failed to send permanent failure notification for {filename}. Error: {notify_e}")
                    logging.debug(f"[DEBUG] Failed to send permanent failure notification.")
                finally:
                    # Update status file to indicate permanent failure (multi-line for file, single for log)
                    permanent_failure_status_file = f"Download Failed Permanently: {filename}\nError: {last_exception}"
                    single_line_permanent_failure_status = permanent_failure_status_file.replace('\n', ', ')
                    logging.error(f"Status file updated: {single_line_permanent_failure_status}")
                    try:
                        with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f_status:
                            f_status.write(permanent_failure_status_file)
                        logging.debug(f"[DEBUG] Permanent failure status written to file.")
                    except Exception as write_e:
                        logging.error(f"Failed to write error shutdown status to file: {write_e}")
                        logging.debug(f"[DEBUG] Failed to write permanent failure status to file.")
                logging.debug(f"[DEBUG] Exiting retry loop after permanent failure.")
                return # Exit the retry loop after permanent failure


# Removed Function to get CPU usage percentage
# Removed Function to get memory usage details
# Removed Function to get CPU temperature (specific to Raspberry Pi/Linux)

# Removed Message Handler for Status Command


# === Message Handler for other incoming messages (downloads) ===
# This handler remains the same as before for processing media
@app.on_message(filters.chat(chat_filter_id) & (filters.document | filters.video | filters.audio | filters.photo))
async def handle_incoming(client, message):
    logging.debug(f"[DEBUG] Entering handle_incoming function for message {message.id}.") # Added debug log
    # Log the incoming message details
    logging.info(f"Received message {message.id} from chat {message.chat.id}. Downloading media...")
    # Removed the call to trigger_emby_scan_from_downloader here
    logging.debug(f"[DEBUG] Calling download_media for message {message.id}.")
    await download_media(client, message) # download_media is already awaited here
    logging.debug(f"[DEBUG] download_media finished for message {message.id}.")


# === Main execution block with KeyboardInterrupt handling ===
logging.info("🟢 Bot running. Waiting for files...")

# Add a line to clear the status file when the script starts or stops cleanly
try:
    status_dir = os.path.dirname(DOWNLOADER_STATUS_FILE)
    if status_dir and status_dir != '' and not os.path.exists(status_dir):
        os.makedirs(status_dir, exist_ok=True)
        logging.info(f"Created status file directory: {status_dir}")

    with open(DOWNLOADER_STATUS_FILE, 'w', encoding='utf-8') as f:
        f.write("Downloader script started.")
    logging.info(f"Initial status written to {DOWNLOADER_STATUS_FILE}")
except Exception as e:
    logging.error(f"Failed to write initial status to file {DOWNLOADER_STATUS_FILE}: {e}")


# Corrected: Added the main function and asyncio.run(main()) call
async def main():
    """Main function to run the Pyrogram client."""
    await app.start()
    logging.info("Pyrogram client started.")
    # Keep the client running indefinitely until interrupted
    await idle() # Use idle() to keep the client alive
    await app.stop()
    logging.info("Pyrogram client stopped.")

# Import idle from pyrogram.methods.utilities.idle
from pyrogram.methods.utilities.idle import idle

if __name__ == "__main__":
    # Added the missing closing parenthesis
    asyncio.run(main())